In [1]:
import numpy
import tensorflow as tf

from model_builder import model_builder, relabel, class_merger, balancer

import model_builder
import tools_keras

from itertools import product

Load chips

In [2]:
specs = {
    'tensor': (
        '../urbangrammar_samba/'
        'spatial_signatures/chips/sample.npz'
    ),
    'folder': "../urbangrammar_samba/spatial_signatures/ai/nw/",
}

In [3]:
data = numpy.load(specs["tensor"], allow_pickle=True)

In [4]:
chips = data["chips"]
labels = data["labels"]

Shuffle data to ensure full randomness before splitting to test, validation and secret.

In [11]:
numpy.random.seed(0)

shuffled_idx = numpy.arange(0, chips.shape[0])
numpy.random.shuffle(shuffled_idx)

chips = chips[shuffled_idx]
labels = labels[shuffled_idx]

Merge groups

In [6]:
group_mapping = [
    ['9_0', '9_1', '9_2', '9_4', '9_5', '2_0', '2_1', '2_2'],
    ['1_0', '3_0', '5_0', '6_0', '8_0'],
    ['0_0', '4_0', '7_0']
]
labels = class_merger(labels, group_mapping)

# If you don't want to merge groups, use `relabel` to ensure your labels are in the proper format
# labels = relabel(labels)

Drop chips from excessive classes

In [7]:
mask = balancer(labels, max_ratio=10)

Total number of selected chips: 18841 out of 65042
Counts:
 {0: 907, 1: 8864, 2: 9070}


In [8]:
labels = labels[mask]
chips = chips[mask]

Rescale

In [9]:
chips = chips / (chips.max() / 255.0)

Split data into train and validation.

In [12]:
batch_size = 32
split_1 = int(chips.shape[0] * 0.6)
split_2 = int(chips.shape[0] * 0.8)


# assert that all labels are present in train and validation datasets
assert (numpy.unique(labels[:split_1]) == numpy.unique(labels[split_1:split_2])).all()
assert (numpy.unique(labels[:split_1]) == numpy.unique(labels[split_2:])).all()


train_dataset = tf.data.Dataset.from_tensor_slices((chips[:split_1], labels[:split_1]))
validation_dataset = tf.data.Dataset.from_tensor_slices((chips[split_1:split_2], labels[split_1:split_2]))
secret_dataset = tf.data.Dataset.from_tensor_slices((chips[split_2:], labels[split_2:]))


train_dataset = train_dataset.batch(batch_size=batch_size)
validation_dataset = validation_dataset.batch(batch_size=batch_size)
secret_dataset = secret_dataset.batch(batch_size=batch_size)

2021-12-13 15:20:17.535031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6684 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:21:00.0, compute capability: 7.5


Create model and fit in a loop over options.

In [ ]:
model_specs = {
    'meta_class_map': group_mapping,
    'meta_class_names': ["centres", "periphery", "countryside"],
    'meta_chip_size': 32,
}

for model_name, bridge, neurons in product(["resnet50", "vgg19", "efficientnet"], ["flatten", "pooling"], [128, 256, 512]):
    model = model_builder.model_builder(
        model_name=model_name, 
        bridge=bridge, 
        top_layer_neurons=neurons,
        n_labels=3,
    )

    h = tools_keras.fit_phase(
        model,
        train_dataset,
        validation_dataset,
        secret_dataset,
        log_folder=specs["folder"] + "logs",
        pred_folder=specs["folder"] + "pred",
        model_folder=specs["folder"] + "model",
        json_folder=specs["folder"] + "json",
        specs=model_specs,
        epochs=250,
        patience=5,
        verbose=False,

    )
    print(model_name, bridge, neurons, "done")

2021-12-13 15:20:39.211599: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 15:20:39.211628: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 15:20:39.211656: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-12-13 15:20:39.275459: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 15:20:39.275662: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2021-12-13 15:20:44.749640: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimiza

INFO:tensorflow:Assets written to: ../urbangrammar_samba/spatial_signatures/ai/nw/model/resnet50_flatten_128_3/assets


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
/workspace/work/signature_ai/tools_keras.py:233: RuntimeWarning: Mean of empty slice.
  wc_top_prob[c] = y_probs[c_id, c].mean()
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


perf_model_accuracy for train: 0.47779548168182373
perf_model_accuracy for val: 0.481157124042511
perf_model_accuracy for secret: 0.49588751792907715
resnet50 flatten 128 done


2021-12-13 15:26:00.533166: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 15:26:00.533197: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 15:26:00.709668: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 15:26:00.709798: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 15:26:02.855316: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 15:26:02.855348: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 15:26:03.426645: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 15:26:03.427551: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 15:26:03.448128: I tensorflow/core/profiler/internal/gpu/cupti_collect

...

2021-12-13 19:14:47.568759: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 19:14:47.568788: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 19:14:47.740268: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 19:14:47.740436: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 19:14:57.065778: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 19:14:57.065813: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 19:14:58.356713: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 19:14:58.369544: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 19:14:58.451360: I tensorflow/core/profiler/internal/gpu/cupti_collect

INFO:tensorflow:Assets written to: ../urbangrammar_samba/spatial_signatures/ai/nw/model/efficientnet_flatten_512_3/assets


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


perf_model_accuracy for train: 0.7561925053596497
perf_model_accuracy for val: 0.6971868276596069
perf_model_accuracy for secret: 0.7028389573097229
efficientnet flatten 512 done


2021-12-13 19:30:46.534367: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 19:30:46.534398: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 19:30:46.726172: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-13 19:30:46.726306: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 19:30:53.053855: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-13 19:30:53.053892: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-13 19:30:54.174464: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-13 19:30:54.180249: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2021-12-13 19:30:54.227415: I tensorflow/core/profiler/internal/gpu/cupti_collect

In [54]:
# tools_keras.flush(specs["folder"])